In [1]:
# autoreload nangs
%reload_ext autoreload
%autoreload 2
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
C:\Users\sensio\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


# Driven Cavity

In this example we solve the two-dimensional incompressible Navier Stokes equations in the Driven Cavity problem

\begin{equation}
    \begin{array}{c}
        \nabla \cdot \mathbf{u} = 0 \\
        \mathbf{u}_t + \mathbf{u} \cdot \nabla \mathbf{u} = \frac{1}{Re} \nabla^2 \mathbf{u} - \nabla p
    \end{array}
\end{equation}

where $\mathbf{u} = (u, v)$ is the velocity of the fluid and $Re$ is the Reynolds number. The geometry of the problem is described as follows

![](https://www.cfd-online.com/W/images/a/a3/Ldc_geom.png)

In [2]:
#imports

import numpy as np 
import matplotlib.pyplot as plt 
import torch
import nangs
from nangs import *

device = "cuda" if torch.cuda.is_available() else "cpu"

nangs.__version__, torch.__version__

('0.1.2', '1.5.0')

In [3]:
U = 1.
Re = 100

class NavierStokes2d(PDE):
    def computePDELoss(self, inputs, outputs):
        
        u, v, p = outputs[:, 0], outputs[:, 1], outputs[:, 2]
        
        # first order derivatives        
        grads = self.computeGrads(u, inputs)       
        dudx, dudy, dudt = grads[:, 0], grads[:, 1], grads[:, 2]
        
        grads = self.computeGrads(v, inputs)       
        dvdx, dvdy, dvdt = grads[:, 0], grads[:, 1], grads[:, 2]
        
        grads = self.computeGrads(p, inputs)       
        dpdx, dpdy = grads[:, 0], grads[:, 1]
            
        # second order derivatives        
        du2dx2 = self.computeGrads(dudx, inputs)[:, 0]       
        dv2dy2 = self.computeGrads(dvdy, inputs)[:, 1]       
        
        # compute losses
        return {
            'mass': dudx + dvdy,
            'mom_x': dudt + u*dudx + v*dudy + dpdx - (1./Re)*(du2dx2 + dv2dy2),
            'mom_y': dvdt + u*dvdx + v*dvdy + dpdy - (1./Re)*(du2dx2 + dv2dy2)
        }

# instanciate pde
pde = NavierStokes2d(inputs=('x', 'y', 't'), outputs=('u', 'v', 'p'))

In [4]:
# mesh 

x = np.linspace(0,1,20)
y = np.linspace(0,1,20)
t = np.linspace(0,10,50)

mesh = Mesh({'x': x, 'y': y, 't': t}, device=device)
pde.set_mesh(mesh)

In [5]:
# initial condition

u0 = np.zeros(len(x)*len(y))
v0 = np.zeros(len(x)*len(y))
p0 = np.zeros(len(x)*len(y))
boco = Dirichlet({'x': x, 'y': y, 't': np.array([0])}, {'u': u0, 'v': v0, 'p': p0}, name='initial', device=device)
pde.add_boco(boco)

In [6]:
# left-right walls

class NeumannX(Neumann):
    def computeBocoLoss(self, inputs, outputs):
        dpdx = self.computeGrads(outputs[:, 2], inputs)[:, 0]
        return {'gradX': dpdx}

u0 = np.zeros(2*len(y)*len(t))
v0 = np.zeros(2*len(y)*len(t))
boco = Dirichlet({'x': np.array([0, 1]), 'y': y, 't': t}, {'u': u0, 'v': v0}, name='x_w_d', device=device)
pde.add_boco(boco)

boco = NeumannX({'x': np.array([0, 1]), 'y': y, 't': t}, name='x_w_n', device=device)
pde.add_boco(boco)

Boco x_w_d with different outputs ! ('u', 'v', 'p') vs ('u', 'v')


In [7]:
# bootom wall

class NeumannY(Neumann):
    def computeBocoLoss(self, inputs, outputs):
        dpdy = self.computeGrads(outputs[:, 2], inputs)[:, 1]
        return {'gradY': dpdy}

u0 = np.zeros(len(x)*len(t))
v0 = np.zeros(len(x)*len(t))
boco = Dirichlet({'x': x, 'y': np.array([0]), 't': t}, {'u': u0, 'v': v0}, name='b_w_d', device=device)
pde.add_boco(boco)

   
boco = NeumannY({'x': x, 'y': np.array([0]), 't': t}, name='b_w_n', device=device)
pde.add_boco(boco)

Boco b_w_d with different outputs ! ('u', 'v', 'p') vs ('u', 'v')


In [8]:
# top wall

u0 = np.full(len(x)*len(t), U)
v0 = np.zeros(len(x)*len(t))
boco = Dirichlet({'x': x, 'y': np.array([1]), 't': t}, {'u': u0, 'v': v0}, name='t_w_d', device=device)
pde.add_boco(boco)

boco = NeumannY({'x': x, 'y': np.array([1]), 't': t}, name='t_w_n', device=device)
pde.add_boco(boco)

Boco t_w_d with different outputs ! ('u', 'v', 'p') vs ('u', 'v')


In [9]:
from nangs import MLP

BATCH_SIZE = 2048
LR = 5e-3
EPOCHS = 50
NUM_LAYERS = 3
NUM_HIDDEN = 256

mlp = MLP(len(pde.inputs), len(pde.outputs), NUM_LAYERS, NUM_HIDDEN).to(device)
optimizer = torch.optim.Adam(mlp.parameters())
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, pct_start=0.1, total_steps=EPOCHS)

pde.compile(mlp, optimizer, scheduler)
hist = pde.solve(EPOCHS, BATCH_SIZE)

KeyboardInterrupt: 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
for name, vale in hist.items():
    if name is not 'lr':
        ax1.plot(hist[name], label=name)
ax1.grid(True)
ax1.legend()
ax1.set_yscale("log")
ax2.plot(hist['lr'], label='lr')
ax2.grid(True)
ax2.legend()
plt.show()

In [ ]:
# evaluate the solution

x = np.linspace(0,1,50)
y = np.linspace(0,1,50)
t = np.linspace(0,10,50)

eval_mesh = Mesh({'x': x, 'y': y, 't': t}, device=device)
outputs = pde.eval(eval_mesh).cpu()

u = outputs[:, 0].view(len(x),len(y),len(t)).numpy()
v = outputs[:, 1].view(len(x),len(y),len(t)).numpy()
p = outputs[:, 2].view(len(x),len(y),len(t)).numpy()

In [ ]:
from matplotlib import animation, rc
rc('animation', html='html5')

def update_plot(i, t, u, v, p):
    _u = u[..., i].reshape(len(y), len(x))
    _v = v[..., i].reshape(len(y), len(x))
    _p = p[..., i].reshape(len(y), len(x))
    vel = np.sqrt(_u**2 + _v**2)
    ax1.clear()
    ax2.clear()
    ax3.clear()
    ax1.imshow(vel, vmin=0, vmax=1, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
    ax1.set_xlabel("x", fontsize=14)
    ax1.set_ylabel("y", fontsize=14, rotation=np.pi/2)
    ax1.set_title(f"t = {t[i]:.2f}", fontsize=14)
    ax1.axis(False)
    ax2.imshow(u, vmin=0, vmax=1, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
    ax3.imshow(v, vmin=0, vmax=1, origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
    ax2.axis(False)
    ax3.axis(False)
    ax4.imshow(p, vmin=p.min(), vmax=p.max(), origin='lower', extent=[x.min(), x.max(), y.min(), y.max()])
    ax4.axis(False)
    return ax1, ax2, ax3, ax4

fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(10,5))
anim = animation.FuncAnimation(fig, update_plot, frames=len(t), fargs=(t, u, v, p), interval=100)
plt.close()

In [ ]:
anim